In [ ]:
import trimesh
import open3d as o3d
import trimesh.exchange.stl

import numpy as np
import matplotlib.pyplot as plt

def plot_trimesh_edges_only(mesh, edge_color=[0, 0, 0, 128]):  # Black with 50% transparency
    """Plot only the edges with specified color and transparency"""
    
    # Create edge visualizations
    edges = mesh.edges_unique
    vertices = mesh.vertices
    
    # Create a collection of lines for each edge
    lines = []
    for edge in edges:
        lines.append(trimesh.path.entities.Line(edge))
    
    # Create colors for each entity (each line) with RGBA
    # Ensure we have 4 values (RGBA) for each color
    if len(edge_color) == 3:
        edge_color = list(edge_color) + [128]  # Add alpha if only RGB provided
    
    colors = np.tile(edge_color, (len(lines), 1))
    
    # Create a Path3D object
    path = trimesh.path.Path3D(
        entities=lines,
        vertices=vertices,
        colors=colors
    )
    
    scene = trimesh.Scene()
    scene.add_geometry(path)
    return scene

# Modify grain geometry from tomography output

In [ ]:
isolated_grains = trimesh.load_mesh('realisticGrains_fromCAD-connected.stl') #realisticGrains_fromPythonOld.stl
# # scaling seems to be in mm, no need to adjust manually. otherwise, use:
# isolated_grains.apply_scale(1e-3)
isolated_grains.show()

In [ ]:
isolated_grains.is_watertight

In [ ]:
# Convert Trimesh → Open3D
mesh_o3d = o3d.geometry.TriangleMesh(
    vertices=o3d.utility.Vector3dVector(isolated_grains.vertices),
    triangles=o3d.utility.Vector3iVector(isolated_grains.faces)
)

# Compute normals (optional but useful for remeshing)
mesh_o3d.compute_vertex_normals()

# Apply Loop subdivision
mesh_remesh = mesh_o3d.subdivide_loop(number_of_iterations=2)

# Convert Open3D → Trimesh
vertices = np.asarray(mesh_remesh.vertices)
faces = np.asarray(mesh_remesh.triangles)
isolated_grains_remeshed = trimesh.Trimesh(vertices=vertices, faces=faces, process=False)

# Visualize with Trimesh
isolated_grains_remeshed.show()

In [ ]:
# The matrix you defined:
flip_transform = np.array([[1, 0, 0, 0],
                           [0, 1, 0, 0],
                           [0, 0, -1, 0], 
                           [0, 0, 0, 1]]) # Added the necessary 4th row for homogeneous coordinates

# CORRECT: Use apply_transform() for matrix operations
isolated_grains_remeshed.apply_transform(flip_transform)

In [ ]:
isolated_grains_remeshed.show()

In [ ]:
centroid = isolated_grains_remeshed.centroid
# Translate the mesh so that the centroid is at (0, 0, 0)
isolated_grains_remeshed.apply_translation(-centroid)

# Get bounding box corners
min_corner, max_corner = isolated_grains_remeshed.bounds  # shape (2, 3)

# Compute size along each axis (X, Y, Z)
size = max_corner - min_corner

print(f"Bounding box min corner: {min_corner}")
print(f"Bounding box max corner: {max_corner}")
print(f"Size (X x Y x Z): {size}")

In [ ]:
isolated_grains_remeshed.apply_translation((0,0,-0.015))

In [ ]:
## determine appropiate grid spacing for Electric Field calculations

# Create mesh grid for exact sampling
WorldX, WorldY, WorldZ = 117-10, 121, 100.29045+15
print(117-10, 121, 100.29045)
stepsize = 10

x_array = np.arange(-WorldX/2, WorldX/2, stepsize)/1000
y_array = np.arange(-WorldY/2, WorldY/2, stepsize)/1000
z_array = np.arange(-WorldZ/2, WorldZ/2, stepsize)/1000

# Create 3D mesh grid
X, Y, Z = np.meshgrid(x_array, y_array, z_array, indexing='ij')

# Flatten the mesh grid to create sampling points
sampling_points = np.column_stack([X.ravel(), Y.ravel(), Z.ravel()])
print(len(sampling_points))
photoelectron_stopping_sites = trimesh.points.PointCloud(sampling_points, colors=[0, 0, 255, 255])

scene = plot_trimesh_edges_only(isolated_grains_remeshed, edge_color=[0, 0, 0, 128])
scene.add_geometry([photoelectron_stopping_sites])
scene.show()

In [ ]:
# Get ASCII STL string
ascii_stl_str = trimesh.exchange.stl.export_stl_ascii(mesh=isolated_grains_remeshed)

# Write it to a file
with open("realisticGrains_fromPython-connected.stl", "w") as f:
    f.write(ascii_stl_str)

# Calculate quantities 

In [ ]:
# ============================================
# VOID VOLUME CALCULATION
# ============================================

# Define the volumetric region dimensions
region_dimensions = np.array([0.11700044, 0.12077438, 0.10029045])*0.8  # X, Y, Z in your units
total_region_volume = np.prod(region_dimensions)

# Calculate the solid volume of the mesh
# Trimesh automatically computes this if the mesh is watertight
if isolated_grains_remeshed.is_watertight:
    solid_volume = isolated_grains_remeshed.volume
    print(f"\nMesh is watertight: Yes")
else:
    # If not watertight, try to fix it or use convex hull as approximation
    print(f"\nMesh is watertight: No (results may be approximate)")
    try:
        isolated_grains_remeshed.fill_holes()
        solid_volume = isolated_grains_remeshed.volume
    except:
        # Fallback: use convex hull volume
        solid_volume = isolated_grains_remeshed.convex_hull.volume
        print("Using convex hull volume as approximation")

# Calculate void volume and percentage
void_volume = total_region_volume - solid_volume
void_percentage = (void_volume / total_region_volume) * 100

# Display results
print(f"\n{'='*50}")
print(f"VOID ANALYSIS RESULTS")
print(f"{'='*50}")
print(f"Region dimensions (X×Y×Z): {region_dimensions[0]:.3f} × {region_dimensions[1]:.3f} × {region_dimensions[2]:.3f}")
print(f"Total region volume:       {total_region_volume:.6f} cubic units")
print(f"Solid geometry volume:     {solid_volume:.6f} cubic units")
print(f"Void volume:               {void_volume:.6f} cubic units")
print(f"Void percentage:           {void_percentage:.2f}%")
print(f"Solid percentage:          {100 - void_percentage:.2f}%")
print(f"{'='*50}")

# Optional: Check if mesh fits within specified region
mesh_fits = np.all(size <= region_dimensions)
print(f"\nMesh fits within specified region: {mesh_fits}")
if not mesh_fits:
    print(f"Warning: Mesh bounding box ({size[0]:.3f}×{size[1]:.3f}×{size[2]:.3f}) exceeds region dimensions!")

In [ ]:

# Create a box centered at origin (since mesh is centered)
half_dims = region_dimensions*0.8 / 2
box_min = -half_dims
box_max = half_dims

# Define the 8 corners of the box
corners = np.array([
    [box_min[0], box_min[1], box_min[2]],
    [box_max[0], box_min[1], box_min[2]],
    [box_max[0], box_max[1], box_min[2]],
    [box_min[0], box_max[1], box_min[2]],
    [box_min[0], box_min[1], box_max[2]],
    [box_max[0], box_min[1], box_max[2]],
    [box_max[0], box_max[1], box_max[2]],
    [box_min[0], box_max[1], box_max[2]]
])

# Define the 12 edges of the box (each edge connects 2 corners)
edges = np.array([
    [0, 1], [1, 2], [2, 3], [3, 0],  # Bottom face
    [4, 5], [5, 6], [6, 7], [7, 4],  # Top face
    [0, 4], [1, 5], [2, 6], [3, 7]   # Vertical edges
])

# Create a Path3D object for the box edges
box_path = trimesh.load_path(corners[edges])

# Set colors correctly - one color per entity (line segment)
num_entities = len(box_path.entities)
box_path.colors = np.array([[255, 0, 0, 255]] * num_entities)  # Red color

# ============================================
# VISUALIZE MESH + BOUNDARY BOX
# ============================================

# Create a scene and add both the mesh and the box
scene = trimesh.Scene([
    isolated_grains_remeshed,
    box_path
])

# Show the combined visualization
scene.show()


In [ ]:
# Areas of all triangles (in the mesh surface)
triangle_areas = isolated_grains_remeshed.area_faces  # shape: (num_faces,)

# Average area
mean_area = triangle_areas.mean()
print(f"Mean triangle area: {mean_area*1000*1000:.8f} nm")
# should translate to an area of around 1 micron... in which case we should not go above a map spacing of 2 microns

plt.hist(triangle_areas*1000*1000, bins=100)
plt.show()